In [ ]:
from tensorflow.keras.preprocessing import image
from keras.models import load_model
from mpl_toolkits.axisartist.axislines import Subplot
from matplotlib import pyplot
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
#import wandb
import matplotlib.image as mpimg

In [ ]:
folder_path = 'inaturalist_12K/val/'

In [ ]:
loaded_model = tf.keras.models.load_model("./model")

In [ ]:


def guidedRelu(input):
  #gradient calculation changes to guided back propagation
  def grad(gradient):
    return tf.cast(gradient>0,"float32") * tf.cast(input>0, "float32") * gradient
  #forward propagation remains unchanged
  return tf.nn.relu(input), grad


In [ ]:



def guided_relu_gradient():
    for layer in loaded_model.layers:
        # if layer is activated by relu activation the back propagation of layer is changed to guided relu
        if hasattr(layer, 'activation'):
            if layer.activation == tf.keras.activations.relu:
                layer.activation = guidedRelu
    loaded_model.compile()



In [ ]:


images=[]
true_image=[]
neuron_no_arr=[50 , 63, 57 , 28,30, 62, 1,6, 8, 43 ]
for i in range (10):
    
# accessing the images of each class 
    for count, sub_folder in enumerate(os.listdir(folder_path)):
        sub_folder_path = folder_path + '/' + sub_folder
        
        for count_1, filename in enumerate(os.listdir(sub_folder_path)):
            name = f"{sub_folder_path}/{filename}"
            loaded_image = image.load_img(name, target_size=(256, 256))
            img_arr = image.img_to_array(loaded_image)
            
            img_numpy = np.expand_dims(img_arr, axis=0)
            
            neuron_output = tf.keras.activations.relu(loaded_model.get_layer('conv2d_4').output[:,:,:,neuron_no_arr[i]])
            neuron = tf.keras.models.Model(inputs = [loaded_model.inputs],outputs = [neuron_output])
            with tf.GradientTape() as tape:
                
                img_tensor=tf.cast(img_numpy,tf.float32)
                
                tape.watch(img_tensor)
                
                output = neuron(img_tensor)
            gradient= tape.gradient(output,img_tensor).numpy()[0]
            
            if np.sum(gradient)>0 :
                images.append(gradient)
                true_image.append(loaded_image)
                break
        
                    
        break 
    

guided_relu_gradient()

fig = pyplot.figure(figsize =(15, 15))
ax = Subplot(fig, 111)
fig.add_subplot(ax)
c=1
for index_1 in  images:
    pyplot.subplot(2,5,c) 
    pyplot.title('by neuron:'+ str(neuron_no_arr[c-1]))
    pyplot.imshow(index_1)
    c+=1

pyplot.show()
c=1
for index_2 in  true_image:
    pyplot.subplot(2,5, c) 
    pyplot.title("by neuron:"+ str(neuron_no_arr[c-1]))
    pyplot.imshow(index_2)
    c+=1
 
pyplot.show()